In [4]:
#dashboard using vtkjs
import panel as pn
import vtk
import pandas as pd
import hvplot.pandas
pn.extension('vtk')

from brainspace.datasets import load_parcellation, load_conte69


orientations = {'medial': (0, -90, -90),
                'lateral': (0, 90, 90),
                'ventral': (0, 180, 0),
                'dorsal': (0, 0, 0)}

    # and load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

def fun1(surf, ori='medial', bg=(1, 1, 1)):

    cylinderMapper = vtk.vtkPolyDataMapper()
    # cylinderMapper.SetInputConnection(cylinder.GetOutputPort())
    cylinderMapper.SetInputData(surf)

    render_actor = vtk.vtkActor()
    render_actor.SetMapper(cylinderMapper)

    cylinderMapper.SetScalarVisibility(0)
    # render_actor.RotateX(30.0)
    # render_actor.RotateY(-45.0)
    render_actor.SetOrientation(orientations[ori])

    ren = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)

    ren.AddActor(render_actor)
    ren.SetBackground(*bg)

    return cylinderMapper, render_actor, ren, renWin

# Schaefer parcellation
labeling = load_parcellation('schaefer', scale=400, join=True)

# load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

#cylinder render
cylinder = vtk.vtkCylinderSource()
cylinder.SetResolution(100)

_, render_actor, ren, renWinLM = fun1(surf_lh.VTKObject, ori='medial', bg=(1, 1, 1))
_, render_actor, ren, renWinLL = fun1(surf_lh.VTKObject, ori='lateral', bg=(1, 1, 1))
_, render_actor, ren, renWinRL = fun1(surf_rh.VTKObject, ori='medial', bg=(1, 1, 1))
# cylinder = surf_rh.VTKObject

# cylinderMapper = vtk.vtkPolyDataMapper()
# # cylinderMapper.SetInputConnection(cylinder.GetOutputPort())
# cylinderMapper.SetInputData(cylinder)

# render_actor = vtk.vtkActor()
# render_actor.SetMapper(cylinderMapper)

# cylinderMapper.SetScalarVisibility(0)
# render_actor.RotateX(30.0)
# render_actor.RotateY(-45.0)
# render_actor.SetOrientation(orientations['medial'])

# ren = vtk.vtkRenderer()
# renWin = vtk.vtkRenderWindow()
# renWin.AddRenderer(ren)

# ren.AddActor(render_actor)
# ren.SetBackground(1.0000, 1.0000, 1.0000)

Lmedial = pn.pane.VTK(renWinLM)
Llateral = pn.pane.VTK(renWinLL)
Rlateral = pn.pane.VTK(renWinRL)

# cyl1.jslink(Rlateral, camera='camera', bidirectional=True)

# if cyl1.camera:
#     cyl1.camera['viewAngle'] = 50
#     cyl1.param.trigger('camera')

# actor initialization for widget 1
actor_ref = ["None", "reference1", "reference2"]
actor_names = ["None", 'Left Medial', 'Right Medial']
actor_opts = {k:v for k,v in zip(actor_names, actor_ref)}

options = {}
actor_selection1 = pn.widgets.Select(value=None, options=actor_opts, margin=0, name="Actor Selection")
actor_selection1

def handle_actor_selection(event):
    selected_actor = event.obj.value
    
    # Update the visibility of the widgets based on the selected actor
    if selected_actor == "Render 1":
        widget1.visible = True
        widget2.visible = False
    elif selected_actor == "Render 2":
        widget1.visible = False
        widget2.visible = True
    else:
        widget1.visible = False
        widget2.visible = False

# Watch the actor selection for changes
actor_selection1.param.watch(handle_actor_selection, 'value')


#Widget for cylinder 1
    #background color 1
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
background_color = pn.widgets.ColorPicker(value=color, name='Background Color')
background_color.jslink(Lmedial, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})

    #opacity 1
opacity = pn.widgets.FloatSlider(value=1, start=0, end=1, name='opacity')
opacity.jscallback(args={"target":Lmedial, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")

#actor link to widget 1
actor_selection1.jslink(target= Lmedial, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"opacity":opacity});

#Rlateral
    #background color 1
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
background_color = pn.widgets.ColorPicker(value=color, name='Background Color')
background_color.jslink(Rlateral, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})

    #opacity 1
opacity = pn.widgets.FloatSlider(value=1, start=0, end=1, name='opacity')
opacity.jscallback(args={"target":Rlateral, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")

#actor link to widget 1
actor_selection1.jslink(target= Rlateral, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"opacity":opacity});


#graph
    #color parse
def hex_to_rgb(hex_code):
    hex_code = hex_code.strip('#')

    red = int(hex_code[0:2], 16)
    green = int(hex_code[2:4], 16)
    blue = int(hex_code[4:6], 16)

    return red, green, blue

def color_picker_callback(event):
    hex_color = event.obj.value
    rgb_color = hex_to_rgb(hex_color)

background_color.param.watch(color_picker_callback, 'value')

#display
widget1 = pn.WidgetBox(background_color, actor_selection1, opacity)
widget2 = pn.WidgetBox(background_color, actor_selection1, opacity)
# tabs = pn.Tabs()
# tabs.extend([
#     ("R Lateral", widget2),
#     ("L Medial", widget1)
# ])

pn.Row(widget1, Lmedial, Rlateral)



BokehModel(combine_events=True, render_bundle={'docs_json': {'e5a75183-a51b-49ee-a94d-97b9590318fd': {'version…

In [5]:
def hex_to_rgb(hex_code):
    # Remove the '#' symbol if present
    hex_code = hex_code.strip('#')

    # Convert the hex code to integer values
    red = int(hex_code[0:2], 16)
    green = int(hex_code[2:4], 16)
    blue = int(hex_code[4:6], 16)

    # Return the RGB tuple
    return red, green, blue

def color_picker_callback(event):
    hex_color = event.obj.value
    rgb_color = hex_to_rgb(hex_color)

color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
Background_Color_Llateral = pn.widgets.ColorPicker(value=color, name='Background Color')
Background_Color_Llateral.param.watch(color_picker_callback, 'value')

Background_Color_Llateral

BokehModel(combine_events=True, render_bundle={'docs_json': {'75e2a4ad-4ab6-4576-a1c3-ca71e8ff3668': {'version…

In [32]:
import panel as pn
import vtk
import pandas as pd
import hvplot.pandas
pn.extension('vtk')

from brainspace.datasets import load_parcellation, load_conte69

orientations = {'medial': (0, -90, -90),
                'lateral': (0, 90, 90),
                'ventral': (0, 180, 0),
                'dorsal': (0, 0, 0)}

#     # and load the conte69 surfaces
# surf_lh, surf_rh = load_conte69()

def fun1(surf, ori='medial', bg=(1, 1, 1)):

    cylinderMapper = vtk.vtkPolyDataMapper()
    # cylinderMapper.SetInputConnection(cylinder.GetOutputPort())
    cylinderMapper.SetInputData(surf)

    render_actor = vtk.vtkActor()
    render_actor.SetMapper(cylinderMapper)

    cylinderMapper.SetScalarVisibility(0)
    # render_actor.RotateX(30.0)
    # render_actor.RotateY(-45.0)
    render_actor.SetOrientation(orientations[ori])

    ren = vtk.vtkRenderer()
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)

    ren.AddActor(render_actor)
    ren.SetBackground(*bg)

    return cylinderMapper, render_actor, ren, renWin

# Schaefer parcellation
labeling = load_parcellation('schaefer', scale=400, join=True)

# load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

_, render_actor, ren, renWinLM = fun1(surf_lh.VTKObject, ori='medial', bg=(1, 1, 1))
_, render_actor, ren, renWinLL = fun1(surf_lh.VTKObject, ori='lateral', bg=(1, 1, 1))
_, render_actor, ren, renWinRL = fun1(surf_rh.VTKObject, ori='medial', bg=(1, 1, 1))
_, render_actor, ren, renWinRM = fun1(surf_rh.VTKObject, ori='lateral', bg=(1, 1, 1))

Lmedial = pn.pane.VTK(renWinLM)
Llateral = pn.pane.VTK(renWinLL)
Rlateral = pn.pane.VTK(renWinRL)
Rmedial = pn.pane.VTK(renWinRM)


#actor initialization
actor_names = ["None", "Left Medial", "Left Lateral", "Right Lateral", "Right Medial"]
actor_selection1 = pn.widgets.Select(options=actor_names, name="Actor Selection")

# Define the callback for actor selection change
def handle_actor_selection(event):
    selected_actor = event.obj.value
    # Update the visibility of the widgets based on the selected actor
    if selected_actor == "Left Medial":
        Background_Color_Lmedial.visible = True
        opacity_Lmedial.visible = True

        Background_Color_Llateral.visible = False
        opacity_Llateral.visible = False

        Background_Color_Rlateral.visible = False
        opacity_Rlateral.visible = False

        Background_Color_Rmedial.visible = False
        opacity_Rmedial.visible = False
    elif selected_actor == "Left Lateral":
        Background_Color_Lmedial.visible = False
        opacity_Lmedial.visible = False

        Background_Color_Llateral.visible = True
        opacity_Llateral.visible = True

        Background_Color_Rlateral.visible = False
        opacity_Rlateral.visible = False

        Background_Color_Rmedial.visible = False
        opacity_Rmedial.visible = False
    elif selected_actor == "Right Lateral":
        Background_Color_Lmedial.visible = False
        opacity_Lmedial.visible = False

        Background_Color_Llateral.visible = False
        opacity_Llateral.visible = False

        Background_Color_Rlateral.visible = True
        opacity_Rlateral.visible = True

        Background_Color_Rmedial.visible = False
        opacity_Rmedial.visible = False

    elif selected_actor == "Right Medial":
        Background_Color_Lmedial.visible = False
        opacity_Lmedial.visible = False

        Background_Color_Llateral.visible = False
        opacity_Llateral.visible = False

        Background_Color_Rlateral.visible = False
        opacity_Rlateral.visible = False

        Background_Color_Rmedial.visible = True
        opacity_Rmedial.visible = True 
    else:
        Background_Color_Lmedial.visible = False
        opacity_Lmedial.visible = False

        Background_Color_Llateral.visible = False
        opacity_Llateral.visible = False

        Background_Color_Rlateral.visible = False
        opacity_Rlateral.visible = False

        Background_Color_Rmedial.visible = False
        opacity_Rmedial.visible = False 
# Watch the actor selection for changes
actor_selection1.param.watch(handle_actor_selection, 'value')


# LMedial widget link
    #background color 1
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
Background_Color_Lmedial = pn.widgets.ColorPicker(value=color, name='Background_Color_Lmedial')
Background_Color_Lmedial.jslink(Lmedial, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})
    #opacity 1
opacity_Lmedial = pn.widgets.FloatSlider(value=1, start=0, end=1, name='Opacity_Lmedial')
opacity_Lmedial.jscallback(args={"target":Lmedial, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")
    #actor link to widget 1
actor_selection1.jslink(target= Lmedial, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"opacity L medial":opacity_Lmedial});



#Llateral link
    #background color 2
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
Background_Color_Llateral = pn.widgets.ColorPicker(value=color, name='Background_Color_Llateral')
Background_Color_Llateral.jslink(Llateral, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})
    #opacity 2
opacity_Llateral = pn.widgets.FloatSlider(value=1, start=0, end=1, name='Opacity_Llateral')
opacity_Llateral.jscallback(args={"target":Llateral, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")
#actor link to widgets for L lateral
actor_selection1.jslink(target= Llateral, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"Opacity L lateral":opacity_Llateral});


#Rlateral link
    #background color 3
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
Background_Color_Rlateral = pn.widgets.ColorPicker(value=color, name='Background_Color_Rlateral')
Background_Color_Rlateral.jslink(Rlateral, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})

    #opacity 3
opacity_Rlateral = pn.widgets.FloatSlider(value=1, start=0, end=1, name='Opacity_Rlateral')
opacity_Rlateral.jscallback(args={"target":Rlateral, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")

    #actor link to widget 3
actor_selection1.jslink(target= Rlateral, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"Opacity_Rlateral":opacity_Rlateral});


# RMedial widget link
    #background color 4
color = ''.join(['#'] + ['{:02x}'.format(int(v*255)) for v in [1, 0, 0]])
Background_Color_Rmedial = pn.widgets.ColorPicker(value=color, name='Background_Color_Rmedial')
Background_Color_Rmedial.jslink(Rmedial, code={'value':"""
const hextoarr = (color) => {return [parseInt(color.slice(1,3),16)/255, parseInt(color.slice(3,5),16)/255, parseInt(color.slice(5,7),16)/255]}
target.renderer_el.getRenderer().setBackground(hextoarr(source.color))

const actor = target.getActors(actor_selection1.value)[0]
actor.getProperty().setopacity(source.value)

target.renderer_el.getRenderWindow().render()
"""})
    #opacity 4
opacity_Rmedial = pn.widgets.FloatSlider(value=1, start=0, end=1, name='Opacity_Rmedial')
opacity_Rmedial.jscallback(args={"target":Rmedial, "actor_selection1":actor_selection1}, value="""
if (actor_selection1.value!="None"){
    const actor = target.getActors(0)[0]
    actor.getProperty().setOpacity(source.value)
    target.renderer_el.getRenderWindow().render()
}
""")
    #actor link to widget 1
actor_selection1.jslink(target= Rmedial, code = {'value': """
if (source.value!="None"){
    const actor = target.getActors(source.value)[0]
    target.renderer_el.getRenderer().addActor(target.outline_actor)
    opacity.setv({value: properties.getopacity()}, {silent: true})
    opacity.disabled = false 
} else {
    target.renderer_el.getRenderer().removeActor(target.outline_actor)
    opacity.disabled = true
}
target.renderer_el.getRenderWindow().render()
"""}, args = {"opacity R medial":opacity_Rmedial});


#display
Lmedial_header = pn.widgets.StaticText(value = "LEFT MEDIAL CONTROLS")
Llateral_header = pn.widgets.StaticText(value = "LEFT LATERAL CONTROLS")
Rlateral_header = pn.widgets.StaticText(value =  "RIGHT LATERAL CONTROLS")
Rmedial_header = pn.widgets.StaticText(value = "RIGHT MEDIAL CONTROLS")

widget1 = pn.WidgetBox(actor_selection1, 
                       Lmedial_header, Background_Color_Lmedial, opacity_Lmedial, 
                       Llateral_header, Background_Color_Llateral, opacity_Llateral, 
                       Rlateral_header, Background_Color_Rlateral, opacity_Rlateral,
                       Rmedial_header, Background_Color_Rmedial, opacity_Rmedial)


# widget1 = pn.WidgetBox(actor_selection1, 
#                     Background_Color_Lmedial, opacity_Lmedial, 
#                     Background_Color_Llateral, opacity_Llateral, 
#                     Background_Color_Rlateral, opacity_Rlateral,
#                     Background_Color_Rmedial, opacity_Rmedial)

grid = pn.GridSpec(sizing_mode = "stretch_both", max_height = 800, max_width = 900)
grid[2, :3] = pn.Spacer(styles={'background': '#0000FF'})
grid[0:2,0] = widget1
grid[0,1] = Lmedial
grid[0,2] = Llateral
grid[1,1] = Rlateral
grid[1,2] = Rmedial
del grid[2, :3]
grid

BokehModel(combine_events=True, render_bundle={'docs_json': {'b8de1fc5-b1c1-4d8a-9cea-0c7c64f187d3': {'version…